In [152]:
import json
import requests
import pandas as pd

from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

In [153]:
with open("../.oauth", "r") as fobj:
     cid, secret = [s.rstrip('\n') for s in fobj.readlines()]

In [154]:
%%capture output
%%bash -s "$cid" "$secret"
curl -u $1:$2 -d grant_type=client_credentials https://classic.warcraftlogs.com/oauth/token

In [157]:
token = json.loads(output.stdout)['access_token']

In [69]:
transport = AIOHTTPTransport(
    url='https://classic.warcraftlogs.com/api/v2/client', 
    headers={'Authorization': 'Bearer ' + token}, 
    timeout=120
)
client = Client(transport=transport, fetch_schema_from_transport=True)

In [73]:
query = """
{
  reportData {
    report(code: "2z3G1AFDbtCXVYcf") {
      guild {
        name
        server
      }
    }
  }
}
"""

In [74]:
res = await client.execute_async(gql(query))

In [75]:
res

{'reportData': {'report': {'guild': {'name': 'Vanguard',
    'server': {'name': 'Azuresong'}}}}}

In [78]:
pd.json_normalize(res["reportData"]["report"], sep="_")

,guild_name,guild_server_name
0,Vanguard,Azuresong
